In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import bs4
from selenium.webdriver.common.by import By
import requests
import os
import shutil # Module offers a number of high-level operations on files and collections of files

## Question 1
###### 1. Get weather information from url: 
###### https://www.accuweather.com/en/in/mumbai/204842/weather-forecast/204842 
###### 1. Make use beautiful soup and selenium.  
###### 2. Extract day, high temperature, low temperature, etc. 

In [2]:
def extract_data_bs4():
    url='https://www.accuweather.com/en/in/mumbai/204842/weather-forecast/204842'
    headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
    r=requests.get(url,headers=headers)
    soup=BeautifulSoup(r.content,'html5lib')
    # print(soup.prettify())
    div=soup.find_all('div',{'class':'today-forecast-card content-module'})
    for d in div:
        date_and_day=d.find('p',{'class':'sub'})
        high_temp = d.find('b').text.split('Hi: ')[1]
        low_temp = d.find_all('b')[1].text.split('Lo: ')[1]

    print(f'Today :{date_and_day.text.strip()}')
    print(f'High_temperature:{high_temp}')
    print(f'LOw_temperature:{low_temp}')

extract_data_bs4()




Today :Sat, Feb 1
High_temperature:30°
LOw_temperature:17°


In [3]:
def extract_data_using_selenium():
    driver=webdriver.Chrome()
    driver.get('https://www.accuweather.com/en/in/mumbai/204842/weather-forecast/204842') # opening the link
    driver.implicitly_wait(5)
    data=driver.find_element(By.XPATH,'/html/body/div/div[7]/div[1]/div[1]/div[1]/a').get_attribute('outerHTML') # getting data in html format
    # data=driver.page_source
    # print(data)
    driver.quit()
    soup=BeautifulSoup(data,'html.parser') # parse thr html
    # print(soup.prettify())
    today=soup.find('p',{'class':'sub'})
    Temp=soup.find_all('b')
    print(f'today :{today.text.strip()}')
    for t in Temp:
        print(t.text)
extract_data_using_selenium()

today :Sat, Feb 1
Hi: 30°
Lo: 17°


## Question 2
###### Customer Threads: 
###### 1. Each customer is represented by a separate thread in the system. 
###### 2. When a customer arrives at the coffee shop, they start a new thread 
###### (representing their order) and place their order.

In [4]:
import random
import threading
import time


def place_order(customer_id):
    
    coffee_types = ['Espresso', 'Latte', 'Cappuccino']
    milk_types = ['Simple Milk', 'Skim Milk', 'Almond Milk']
    sizes = ['Small', 'Medium', 'Large']
    coffee = random.choice(coffee_types)
    milk = random.choice(milk_types)
    size = random.choice(sizes)

   
    print(f"Customer {customer_id} is ordering a {size} {coffee} with {milk}.")
    
    
    time.sleep(4)
    
    print(f"Customer {customer_id}'s {size} {coffee} with {milk} is ready! (Took {4} seconds)")


def simulate_coffee_shop(num_customers):
    threads = []

    
    for customer_id in range(1, num_customers + 1):
        thread = threading.Thread(target=place_order, args=(customer_id,))
        threads.append(thread)
        thread.start()  # Start the thread

   
    for thread in threads:
        thread.join()



simulate_coffee_shop(5)




Customer 1 is ordering a Small Latte with Almond Milk.
Customer 2 is ordering a Medium Latte with Skim Milk.
Customer 3 is ordering a Large Cappuccino with Almond Milk.
Customer 4 is ordering a Medium Cappuccino with Simple Milk.
Customer 5 is ordering a Small Latte with Almond Milk.
Customer 3's Large Cappuccino with Almond Milk is ready! (Took 4 seconds)
Customer 1's Small Latte with Almond Milk is ready! (Took 4 seconds)
Customer 4's Medium Cappuccino with Simple Milk is ready! (Took 4 seconds)
Customer 5's Small Latte with Almond Milk is ready! (Took 4 seconds)
Customer 2's Medium Latte with Skim Milk is ready! (Took 4 seconds)


## Question 3
###### Following is the url where it generates new picture every time when got hit. 
###### 1. https://picsum.photos/2000/3000 
###### 2. Use this url and download images 5 times using multiprocessing module 
###### 3. Check execution time with and without multiprocessing

In [ ]:
# Using Multithreading Download Images
cd=os.getcwd()
cd=cd+"/output/"
url='https://stocksnap.io/search/nature'
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
r=requests.get(url,headers=headers)
soup=BeautifulSoup(r.content,'html5lib')
# print(soup.prettify())
data=soup.find_all('img')
urls=data[20:]
# print(urls)
def save_image(a):
    for a in urls:
        image=a.get('src')
        print(image)
        file_name=a.get('src').split("/")[-1]
        data=requests.get(image,stream=True)
        data.raw.decode_content=True
        with open(cd+file_name,'w+b') as f1:
            shutil.copyfileobj(data.raw,f1)
            f1.close()
        print(file_name)

for i in range(len(urls)):
    save_image(urls[i])






In [2]:
from PIL import Image
from multiprocessing import Pool
import time

In [ ]:
from PIL import Image
from multiprocessing import Pool
import requests
import time
# URL for downloading the images
url = "https://picsum.photos/2000/3000"
 
# Function to download an image
def download_image(index):
    response = requests.get(url)
    print(response)
    if response.status_code == 200:
        with open(f"./output/image_{index}.jpg", "wb") as file:
            file.write(response.content)
        print(f"Downloaded image_{index}.jpg")

 
# # Without multiprocessing (sequential download)
def download_images_sequential():  # Took 12.72 Second
    start_time = time.time()
    for i in range(2):
        download_image(i)
    end_time = time.time()
    print(f"Sequential download took: {end_time - start_time:.2f} seconds")

 
# # With multiprocessing (download)
def download_images_multiprocessing():
    start_time = time.time()
    p=Pool()
    l=[x for x in range(5)]
    p.map(download_image,l)
    # print(data)
    end_time = time.time()
    print(f"Multiprocessing download took: {end_time - start_time:.2f} seconds")


 
# # Run both functions to compare the execution times
print("Starting sequential download...")
download_images_sequential()
 
print("\nStarting multiprocessing download...")
download_images_multiprocessing()

Starting sequential download...
<Response [200]>
Downloaded image_0.jpg
<Response [200]>
Downloaded image_1.jpg
Sequential download took: 8.19 seconds

Starting multiprocessing download...
